In [1]:
import xagg as xa
import geopandas as gpd 
import xarray as xr
import pandas as pd
import pyreadstat

In [2]:
df_paths = pd.read_csv("car_paths.csv", dtype=str)

df_paths.head()

,product,filepath
0,ERA5-025,/shared/share_hle/data/climate_raw/ERA5-025/ta...
1,GMFD,/shared/share_hle/data/climate_raw/GMFD/tas_da...
2,MERRA2,/shared/share_hle/data/climate_raw/MERRA2/tas_...
3,JRA-3Q,/shared/share_hle/data/climate_raw/JRA-3Q/tas_...


In [6]:
ds = xr.open_zarr(df_paths['filepath'].iloc[0], consolidated=False)

In [7]:
print(ds.time)

<xarray.DataArray 'time' (time: 16071)> Size: 129kB
array(['1981-01-01T00:00:00.000000000', '1981-01-02T00:00:00.000000000',
       '1981-01-03T00:00:00.000000000', ..., '2024-12-29T00:00:00.000000000',
       '2024-12-30T00:00:00.000000000', '2024-12-31T00:00:00.000000000'],
      shape=(16071,), dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 129kB 1981-01-01 1981-01-02 ... 2024-12-31
Attributes:
    long_name:  time


In [3]:
df = df_paths.copy()

In [4]:
shape_path = "/shared/share_hle/data/aux_data/geo_data/impact-region.shp"

gdf = gpd.read_file(shape_path)

/user/ab5405/.conda/envs/hle_iv/lib/python3.12/site-packages/pyogrio/raw.py:198: RuntimeWarning: /shared/share_hle/data/aux_data/geo_data/impact-region.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(


In [5]:
# 1) Number of regions × number of columns
print("Shape:", gdf.shape)         # e.g. (n_regions, n_fields)

# 2) List all columns
print("Columns:", gdf.columns.tolist())

# 3) CRS (coordinate reference system)
print("CRS:", gdf.crs)

# 4) Bounding box of all geometries [minx, miny, maxx, maxy]
print("Total bounds:", gdf.total_bounds)

# 5) Geometry types (should all be Polygon or MultiPolygon)
print("Geom types:", gdf.geometry.geom_type.unique())

# 6) Memory footprint
print("Memory usage:\n", gdf.memory_usage(deep=True))

Shape: (24378, 7)
Columns: ['gadmid', 'hierid', 'color', 'ISO', 'AREA', 'PERIMETER', 'geometry']
CRS: EPSG:4326
Total bounds: [-180.00001526  -90.          180.           83.62741852]
Geom types: ['Polygon' 'MultiPolygon']
Memory usage:
 Index            132
gadmid        195024
hierid       1541437
color         195024
ISO          1267656
AREA          195024
PERIMETER     195024
geometry      195024
dtype: int64


In [ ]:
ds = ds.assign(T1 = ds.tas, T2 = ds.tas**2, T3 = ds.tas**3,T4 = ds.tas**4)

# 4) Sum over days and take the mean
T1_sum = ds.T1.sum("time")
T2_sum = ds.T2.sum("time")
T3_sum = ds.T3.sum("time")
T4_sum = ds.T4.sum("time")
Tmean  = ds.tas.mean("time")

In [ ]:
def open_climate(ds_path):
    if ds_path.endswith(".zarr"):
        return xr.open_zarr(ds_path, consolidated=False)
    else:
        return xr.open_dataset(ds_path)



In [ ]:
xa.set_options(impl="numba", silent=True)

all_rows = []

years = range(1957, 2014)

for i, row in df.iterrows():
    ds = open_climate(row['filepath'])
    
    product = row["product"]
    path     = row["filepath"]
    print(f"Processing {product}")
    
    wm      = xa.pixel_overlaps(ds_grid, gdf)
    
    for yr in years:
        ds_yr = ds_all.sel(time=slice(f"{yr}-01-01", f"{yr}-12-31"))

        ds_poly = xr.Dataset({
            "T1_sum": (ds_yr.tas**1).sum("time"),
            "T2_sum": (ds_yr.tas**2).sum("time"),
            "T3_sum": (ds_yr.tas**3).sum("time"),
            "T4_sum": (ds_yr.tas**4).sum("time"),
            "Tmean" :  ds_yr.tas.mean("time"),
        })

        agg    = xa.aggregate(ds_poly, wm)
        df_reg = agg.to_dataframe().reset_index()

        df_reg["product"] = product
        df_reg["year"]    = yr

        all_rows.append(df_reg)

big = pd.concat(all_rows, ignore_index=True)

In [ ]:
big = big.rename(columns={"region_id": "region"})
if "time" in big.columns:
    big = big.drop(columns="time")

cols = ["product", "region", "year"] + [c for c in big.columns if c not in ("product","region","year")]
big  = big[cols]

out_path = "/mnt/data/all_products_by_region_year.dta"
big.to_stata(out_path, write_index=False, version=118)
print(f"Wrote {out_path}")